# Create csv tables from the database

In this notebook, we extract the data by taking the tables in sql converting to csvs for subsequent analysis. Here we will merge the information that we have on current ICUs. We will also extract the data for various sensitivity analyses.

It is important to note that in our study, this part of the process was not transparent to those developing the models, acting as a "blind experiment". This is why later in this notebook we will map the ids found in MIMIC to another set of ids to make reconstruction harder.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

In [2]:
pd.set_option('display.max_columns', None)

### Set up data access

In [3]:
sqluser = 'mimicuser'
dbname = 'mimic'

In [4]:
schema_name = 'mimiciii'
query_schema = 'SET search_path to public,' + schema_name + ';'

In [5]:
import getpass
password = getpass.getpass() 

········


In [6]:
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, host="localhost", password=password)

Set up some useful functions to aid working with sql and  to summarise our exclusions later

In [7]:
def load_from_sql(script, con, schema_name):
        # Read the SQL query from the provided sql script
        f = open(script, 'r')
        sql_query = f.read()
        f.close()
        df = execute_query(sql_query, con, schema_name)
        return df

def execute_query(query, con, schema_name):
    cur = con.cursor()
    cur.execute('SET search_path to ' + schema_name)
    return pd.read_sql_query(query,con)

In [8]:
def sepsis_summary(df_sepsis):

    times = ['t_suspicion', 't_sofa', 't_sepsis_min', 't_sepsis_max']
    
    print(f'There are {len(df_sepsis.icustay_id.unique())} ICU stays after LOS, vital sign and stay exclusions')
    
    for time in times:
        print(f'---------------\n Looking at {time}')
        
        len_remove = len(df_sepsis[df_sepsis[time] < df_sepsis.intime+pd.Timedelta(hours=4)]\
            .icustay_id.unique())

        len_keep = len(df_sepsis[df_sepsis[time].between(df_sepsis.intime+pd.Timedelta(hours=4),\
                                                df_sepsis.outtime, inclusive=True)].icustay_id.unique())

        print(f'We will be keeping {len_keep} ICU stays and removing {len_remove}\n')
 

### Create a current ICU column

We start by extracting vital sign information of the training data and merge with the current ICU the patients are in. To do so, we need to make sure that each patient has a uniquely defined position at any one time. So let us check the information from our current_icu view (see the `current_icu.sql` file for how this view is defined)

In [9]:
Q = """select icustay_id, charttime, curr_careunit 
from current_icu c 
order by icustay_id, charttime"""
df_care = execute_query(Q, con, schema_name)

Remove those entries that are not linked to an icustay_id

In [10]:
df_care = df_care[~df_care.icustay_id.isnull()]

The following table shows that we have duplicated rows

In [11]:
df_care[df_care.duplicated(keep= False)].head()

,icustay_id,charttime,curr_careunit
92964,200817.0,2116-07-15 13:11:00,CSRU
92965,200817.0,2116-07-15 13:11:00,CSRU
92966,200817.0,2116-07-15 13:15:00,CSRU
92967,200817.0,2116-07-15 13:15:00,CSRU
92968,200817.0,2116-07-15 13:30:00,CSRU


We can filter these out

In [12]:
df_care = df_care.drop_duplicates()

In [13]:
df_care = df_care.sort_values(by=['icustay_id','charttime','curr_careunit'], ignore_index=True)

However there are still repeated data for some patient at a particular charttime

In [14]:
df_repeats = df_care[df_care[['icustay_id', 'charttime']].duplicated(keep= False)]

df_repeats = df_repeats.sort_values(by=['icustay_id','charttime','curr_careunit'])

In [15]:
len(df_repeats)

636


This is actually caused by conflicting information for 8 patients in the original transfer data. We shall look at each of these patients closely to determine which location they are more likely to be.

In [16]:
df_repeats.icustay_id.unique()

array([200182., 208656., 208996., 241787., 245640., 275357., 278029.,
       281227.])

We can look at the original information from the transfer table. Let's look at 275357 as an example

In [17]:
df_repeats[df_repeats.icustay_id == 275357].head()

,icustay_id,charttime,curr_careunit
8065440,275357.0,2184-03-20 01:00:00,NICU
8065441,275357.0,2184-03-20 01:00:00,NWARD
8065442,275357.0,2184-03-20 13:10:00,NICU
8065443,275357.0,2184-03-20 13:10:00,NWARD
8065444,275357.0,2184-03-21 06:00:00,NICU


In [18]:
Q = """select * from transfers where icustay_id = 275357"""
test = execute_query(Q, con, schema_name)

In [19]:
test

,row_id,subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,prev_wardid,curr_wardid,intime,outtime,los
0,43664,8017,131067,275357,carevue,admit,None,NICU,NaN,26,2184-03-18 01:20:00,2184-03-18 12:31:00,11.18
1,43665,8017,131067,275357,carevue,transfer,NICU,NICU,26.0,26,2184-03-18 12:31:00,2184-03-22 13:50:00,97.32


By looking at tables like above for all patients, we can make a judgement as to which duplicate to keep. After looking at the raw data, we will get rid of the CCU repeats for 281227, CSRU repeats for 200182, Get rid of NWARD for 275357 and the None repeats for the others. To do this, we compile a list of all the indices that we want to remove by keeping either first or last as appropriate

In [20]:
index1 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'last'))\
        &(df_care.icustay_id==281227)
index2 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'last'))\
        &(df_care.icustay_id==200182)
index3 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==208656)
index4 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==245640)
index5 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==278029)
index6 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==208996)
index7 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==241787)
index8 = (df_care[['icustay_id', 'charttime']].duplicated(keep= 'first'))\
        &(df_care.icustay_id==275357)

In [21]:
drop1 = np.where(index1)[0]
drop2 = np.where(index2)[0]
drop3 = np.where(index3)[0]
drop4 = np.where(index4)[0]
drop5 = np.where(index5)[0]
drop6 = np.where(index6)[0]
drop7 = np.where(index7)[0]
drop8 = np.where(index8)[0]

In [22]:
all_dropped_index = np.concatenate((drop1, drop2, drop3, drop4, drop5, drop6, drop7, drop8))

In [23]:
len(all_dropped_index)

318

Let us do a sanity check first by checking that this works for the "drop1" indices

In [24]:
drop1

array([8706731, 8706733, 8706735, 8706737, 8706739, 8706741, 8706743,
       8706745, 8706747, 8706749, 8706751, 8706753, 8706755, 8706757,
       8706759, 8706761, 8706763, 8706765, 8706767, 8706769, 8706771,
       8706773, 8706775, 8706777, 8706779, 8706781, 8706783, 8706785,
       8706787, 8706789, 8706791, 8706793, 8706795, 8706797, 8706799,
       8706801, 8706803, 8706805, 8706807, 8706809, 8706811, 8706813,
       8706815, 8706817, 8706819, 8706821, 8706823, 8706825, 8706827,
       8706829, 8706831, 8706833, 8706835, 8706837, 8706839, 8706841])

We can see that originally we have duplicate entries of CSRU and CCU.

In [25]:
df_care.iloc[8706731:8706735]

,icustay_id,charttime,curr_careunit
8706731,281227.0,2134-04-20 10:00:00,CCU
8706732,281227.0,2134-04-20 10:00:00,CSRU
8706733,281227.0,2134-04-20 11:00:00,CCU
8706734,281227.0,2134-04-20 11:00:00,CSRU


But if we dropped the index, then we remove all CCU entries as desired

In [26]:
df_care.drop(df_care.index[all_dropped_index]).iloc[8706469:8706472]

,icustay_id,charttime,curr_careunit
8706732,281227.0,2134-04-20 10:00:00,CSRU
8706734,281227.0,2134-04-20 11:00:00,CSRU
8706736,281227.0,2134-04-20 11:29:00,CSRU


In [27]:
df_care = df_care.drop(df_care.index[all_dropped_index])

Now we verify that there are no duplicates after dropping these indices.

In [28]:
df_care[df_care[['icustay_id', 'charttime']].duplicated()]

,icustay_id,charttime,curr_careunit


### Remapping ids

For our experiments, we have used an additional 1 to 1 map for the subject_ids, hadm_ids, and icustay_ids found in MIMIC to a random key (the mapping is found in the three files that we load below). These keys were made by a random permutation of 1 to len(ids) for each of subject_id, hadm_id and icustay_id. This part of the pipeline is to obscure the original information from the members of the team that will be running the models so that it is harder to construct which patients will be in the test set, ensuring fairness in our set-up.

We load the mapping here.

In [29]:
df_subject_ids = pd.read_csv('./mapping_keys/subject_key_combined.csv')
df_hadm_ids = pd.read_csv('./mapping_keys/hadm_key_combined.csv')
df_icustay_ids = pd.read_csv('./mapping_keys/icustay_key_combined.csv')

### Sepsis - Blood cultures only

We extract the data of the patients in the training set for the sepsis time that is computed with a SOFA window of 48 hours before and 24 hours after time of suspected infection.

Note that we decided later to merge the training and validation set as we switched to cross-validation, and therefore we extract all data pertaining to train and val here.

In [30]:
Q = """
with ids as (select * from train_ids union select * from val_ids)
select e.*, s.t_suspicion, s.t_sofa, s.t_sepsis_min, s.t_sepsis_max
from extracted_data e 
inner join ids on e.subject_id = ids.subject_id
  inner join sepsis_cohort_time_blood_sensitivity_4824 s 
  on e.icustay_id = s.icustay_id order by subject_id, charttime"""
df_sepsis = execute_query(Q, con, schema_name)

There appears to be some duplicated entries that we will remove

In [31]:
len(df_sepsis[df_sepsis.duplicated()])

206

In [32]:
df_sepsis = df_sepsis.drop_duplicates()

However, it appears that there are still duplicate times for a few patients.

In [33]:
len(df_sepsis[df_sepsis[['icustay_id', 'charttime']].duplicated(keep=False)])

22

We check for possible explanations by looking at the results returned by the following queries

In [34]:
Q = """select * from extracted_chart where icustay_id = 287417
order by charttime
"""
test = execute_query(Q, con, schema_name)

In [35]:
test.hadm_id.unique()

array([193734, 157999])

In [36]:
Q = """select * from chartevents where icustay_id = 287417
order by charttime
"""
test = execute_query(Q, con, schema_name)

In [37]:
test.hadm_id.unique()

array([193734, 157999])

In [38]:
Q = """select * from extracted_chart where hadm_id = 157999
order by charttime
"""
test = execute_query(Q, con, schema_name)

In [39]:
Q = """select * from extracted_data where hadm_id = 157999
order by charttime
"""
test = execute_query(Q, con, schema_name)

After some inspection, it appears that the above problems seem to be caused by having spurious hadm_id matched with icustay_id for blood glucose readings, perhaps caused some admin error? However for the purposes of our research these would not have much impact given that we are taking hourly summary of the data. If our data format changes, then this issue will have to be addressed.

In [40]:
df_sepsis_merge = df_sepsis.merge(df_care, on=['icustay_id', 'charttime'], how='inner')

Next we remap the ids to our random mapping as explained above.

In [41]:
print('Merging new icustay_ids')

df_newid2 = pd.merge(df_icustay_ids, df_sepsis_merge, on='icustay_id')

print('Merging new hadm_ids')

df_newid2 = pd.merge(df_hadm_ids, df_newid2, on='hadm_id')

print('Merging new subject_ids')

df_newid2 = pd.merge(df_subject_ids, df_newid2, on='subject_id')

Merging new icustay_ids
Merging new hadm_ids
Merging new subject_ids


In [42]:
df_newid2.drop(['subject_id', 'hadm_id', 'icustay_id'], axis = 1, inplace=True)

df_newid2.rename(columns={"subject_id_random": "subject_id", "hadm_id_random":\
                          "hadm_id", "icustay_id_random":"icustay_id"}, inplace=True)

Next we reorder the columns

In [43]:
df_newid2.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'admission_type', 'admittime',
       'dischtime', 'hospital_expire_flag', 'deathtime', 'intime', 'outtime',
       ...
       'sofa_liver', 'sofa_circu', 'sofa_cns', 'sofa_renal', 'sofa_total',
       't_suspicion', 't_sofa', 't_sepsis_min', 't_sepsis_max',
       'curr_careunit'],
      dtype='object', length=116)

In [44]:
static_info = ['subject_id', 'hadm_id', 'icustay_id', 'admission_type', 'admittime',
               'dischtime', 'hospital_expire_flag', 'deathtime', 'intime', 'outtime', 'admit_diff', 
               'age', 'gender', 'ethnicity', 'insurance', 'initial_diagnosis', 'first_careunit',
               'last_careunit', 'dbsource', 'los_hospital', 'hospstay_seq', 'los_icu',
               'icustay_seq', 't_suspicion', 't_sofa', 't_sepsis_min', 't_sepsis_max']

In [45]:
new_columns = static_info + (df_newid2.columns.drop(static_info).tolist())

df_newid2 = df_newid2[new_columns]

df_sepsis = df_newid2.sort_values(['subject_id', 'charttime'])

We see that almost half of the patient are classed as will be having sepsis at some point

In [46]:
len(df_sepsis[~df_sepsis.t_sepsis_min.isnull()].icustay_id.unique())

6330

In [47]:
len(df_sepsis.icustay_id.unique())

13971

Exclude patients whose stay was less than 4 hours and greater than 20 days

In [48]:
df_sepsis = df_sepsis[(df_sepsis['outtime'] <= df_sepsis['intime']+pd.Timedelta(days=20)) & \
             (df_sepsis['outtime'] >= df_sepsis['intime']+pd.Timedelta(hours=4))]

In [49]:
len(df_sepsis.icustay_id.unique())

13704

In [50]:
# We want to filter out patients without any vital sign data

vitals = ['heart_rate', 'nbp_sys', 'nbp_dias', 'nbp_mean', 'abp_sys',
       'abp_dias', 'abp_mean', 'temp_celcius', 'o2sat', 'resp_rate']

df_vitals = df_sepsis[['icustay_id']+vitals].groupby(by='icustay_id', as_index=False).max()
df_max = df_vitals.drop(['icustay_id'], axis=1).max(axis=1)
keep_ids = df_vitals[~df_max.isnull()].icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [51]:
len(df_sepsis.icustay_id.unique())

13604

Next we keep only the index hospital admission of each patient

In [52]:
df_info = df_sepsis[['subject_id', 'icustay_id', 'intime']].drop_duplicates()
df_info.sort_values('intime')
df_info2 = df_info.groupby(by='subject_id', as_index=False).first()
keep_ids = df_info2.icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

Next we give a summary of what would happen in the next stage of the analysis, when we make further exclusions based on whether the patient's time of sepsis is within 4 hours of ICU admission.

In [53]:
sepsis_summary(df_sepsis)

There are 11270 ICU stays after LOS, vital sign and stay exclusions
---------------
 Looking at t_suspicion
We will be keeping 1861 ICU stays and removing 3079

---------------
 Looking at t_sofa
We will be keeping 1220 ICU stays and removing 3720

---------------
 Looking at t_sepsis_min
We will be keeping 776 ICU stays and removing 4164

---------------
 Looking at t_sepsis_max
We will be keeping 2305 ICU stays and removing 2635



In [54]:
df_sepsis.to_csv('../../data/raw/blood_only_sensitivity_48_24.csv',\
                 index=False)

##  Other SOFA windows

We process the data for the other SOFA windows (24,12), (12, 6), (6,3) in the same way

#### SOFA window: (24,12)

In [55]:
Q = """
with ids as (select * from train_ids union select * from val_ids)
select e.*, s.t_suspicion, s.t_sofa, s.t_sepsis_min, s.t_sepsis_max
from extracted_data e 
inner join ids on e.subject_id = ids.subject_id
  inner join sepsis_cohort_time_blood_sensitivity_2412 s 
  on e.icustay_id = s.icustay_id order by subject_id, charttime"""
df_sepsis = execute_query(Q, con, schema_name)

In [56]:
df_sepsis = df_sepsis.drop_duplicates()

Again, we have a few cases of duplicated times, this will be resolved later in our pipeline when we bin the data on an hourly basis. 

In [57]:
len(df_sepsis[df_sepsis[['icustay_id', 'charttime']].duplicated()])

11

In [58]:
df_sepsis_merge = df_sepsis.merge(df_care, on=['icustay_id', 'charttime'], how='inner')

Next we again remap the ids

In [59]:
print('Merging new icustay_ids')

df_newid2 = pd.merge(df_icustay_ids, df_sepsis_merge, on='icustay_id')

print('Merging new hadm_ids')

df_newid2 = pd.merge(df_hadm_ids, df_newid2, on='hadm_id')

print('Merging new subject_ids')

df_newid2 = pd.merge(df_subject_ids, df_newid2, on='subject_id')

Merging new icustay_ids
Merging new hadm_ids
Merging new subject_ids


In [60]:
df_newid2.drop(['subject_id', 'hadm_id', 'icustay_id'], axis = 1, inplace=True)

df_newid2.rename(columns={"subject_id_random": "subject_id", "hadm_id_random": "hadm_id",\
                          "icustay_id_random":"icustay_id"}, inplace=True)

We move the static columns to the start of the dataframe

In [61]:
new_columns = static_info + (df_newid2.columns.drop(static_info).tolist())

df_newid2 = df_newid2[new_columns]

df_sepsis = df_newid2.sort_values(['subject_id', 'charttime'])

We see that fewer patients have been classed as having sepsis compared with previously

In [62]:
len(df_sepsis[~df_sepsis.t_sepsis_min.isnull()].icustay_id.unique())

5963

In [63]:
len(df_sepsis.icustay_id.unique())

13971

Exclude patients whose stay was less than 4 hours and greater than 20 days, then give a summary

In [64]:
df_sepsis = df_sepsis[(df_sepsis['outtime']<=df_sepsis['intime']+pd.Timedelta(days=20)) & \
             (df_sepsis['outtime']>=df_sepsis['intime']+pd.Timedelta(hours=4))]

In [65]:
len(df_sepsis.icustay_id.unique())

13704

In [66]:
# We want to filter out patients without any vital sign data

vitals = ['heart_rate', 'nbp_sys', 'nbp_dias', 'nbp_mean', 'abp_sys',
       'abp_dias', 'abp_mean', 'temp_celcius', 'o2sat', 'resp_rate']

df_vitals = df_sepsis[['icustay_id']+vitals].groupby(by='icustay_id', as_index=False).max()
df_max = df_vitals.drop(['icustay_id'], axis=1).max(axis=1)
keep_ids = df_vitals[~df_max.isnull()].icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [67]:
len(df_sepsis.icustay_id.unique())

13604

Next we keep only the index hospital admission of each patient

In [68]:
df_info = df_sepsis[['subject_id', 'icustay_id', 'intime']].drop_duplicates()
df_info.sort_values('intime')
df_info2 = df_info.groupby(by='subject_id', as_index=False).first()
keep_ids = df_info2.icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [69]:
sepsis_summary(df_sepsis)

There are 11270 ICU stays after LOS, vital sign and stay exclusions
---------------
 Looking at t_suspicion
We will be keeping 1733 ICU stays and removing 2901

---------------
 Looking at t_sofa
We will be keeping 1117 ICU stays and removing 3517

---------------
 Looking at t_sepsis_min
We will be keeping 815 ICU stays and removing 3819

---------------
 Looking at t_sepsis_max
We will be keeping 2035 ICU stays and removing 2599



In [70]:
df_sepsis.to_csv('../../data/raw/blood_only_sensitivity_24_12.csv',\
                 index=False)

#### SOFA window: (12,6)

In [71]:
Q = """
with ids as (select * from train_ids union select * from val_ids)
select e.*, s.t_suspicion, s.t_sofa, s.t_sepsis_min, s.t_sepsis_max
from extracted_data e 
inner join ids on e.subject_id = ids.subject_id
  inner join sepsis_cohort_time_blood_sensitivity_126 s 
  on e.icustay_id = s.icustay_id order by subject_id, charttime"""
df_sepsis = execute_query(Q, con, schema_name)

In [72]:
df_sepsis = df_sepsis.drop_duplicates()

Note the same issue with a few duplicate chartime points remains

In [73]:
len(df_sepsis[df_sepsis[['icustay_id', 'charttime']].duplicated()])

11

In [74]:
df_sepsis_merge = df_sepsis.merge(df_care, on=['icustay_id', 'charttime'], how='inner')

Next we again remap the ids

In [75]:
print('Merging new icustay_ids')

df_newid2 = pd.merge(df_icustay_ids, df_sepsis_merge, on='icustay_id')

print('Merging new hadm_ids')

df_newid2 = pd.merge(df_hadm_ids, df_newid2, on='hadm_id')

print('Merging new subject_ids')

df_newid2 = pd.merge(df_subject_ids, df_newid2, on='subject_id')

Merging new icustay_ids
Merging new hadm_ids
Merging new subject_ids


In [76]:
df_newid2.drop(['subject_id', 'hadm_id', 'icustay_id'], axis = 1, inplace=True)

df_newid2.rename(columns={"subject_id_random": "subject_id", "hadm_id_random": "hadm_id",\
                          "icustay_id_random":"icustay_id"}, inplace=True)

We move the static columns to the start of the dataframe

In [77]:
new_columns = static_info + (df_newid2.columns.drop(static_info).tolist())

df_newid2 = df_newid2[new_columns]

df_sepsis = df_newid2.sort_values(['subject_id', 'charttime'])

Again we see that the number of people with sepsis is decreasing with a decreased window size

In [78]:
len(df_sepsis[~df_sepsis.t_sepsis_min.isnull()].icustay_id.unique())

5279

In [79]:
len(df_sepsis.icustay_id.unique())

13971

Exclude patients based on length of stay and print summary for future exclusions

In [80]:
df_sepsis = df_sepsis[(df_sepsis['outtime']<=df_sepsis['intime']+pd.Timedelta(days=20)) & \
             (df_sepsis['outtime']>=df_sepsis['intime']+pd.Timedelta(hours=4))]

In [81]:
len(df_sepsis.icustay_id.unique())

13704

In [82]:
# We want to filter out patients without any vital sign data

vitals = ['heart_rate', 'nbp_sys', 'nbp_dias', 'nbp_mean', 'abp_sys',
       'abp_dias', 'abp_mean', 'temp_celcius', 'o2sat', 'resp_rate']

df_vitals = df_sepsis[['icustay_id']+vitals].groupby(by='icustay_id', as_index=False).max()
df_max = df_vitals.drop(['icustay_id'], axis=1).max(axis=1)
keep_ids = df_vitals[~df_max.isnull()].icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [83]:
len(df_sepsis.icustay_id.unique())

13604

Next we keep only the index hospital admission of each patient

In [84]:
df_info = df_sepsis[['subject_id', 'icustay_id', 'intime']].drop_duplicates()
df_info.sort_values('intime')
df_info2 = df_info.groupby(by='subject_id', as_index=False).first()
keep_ids = df_info2.icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [85]:
sepsis_summary(df_sepsis)

There are 11270 ICU stays after LOS, vital sign and stay exclusions
---------------
 Looking at t_suspicion
We will be keeping 1514 ICU stays and removing 2562

---------------
 Looking at t_sofa
We will be keeping 941 ICU stays and removing 3135

---------------
 Looking at t_sepsis_min
We will be keeping 823 ICU stays and removing 3253

---------------
 Looking at t_sepsis_max
We will be keeping 1632 ICU stays and removing 2444



In [86]:
df_sepsis.to_csv('../../data/raw/blood_only_sensitivity_12_6.csv',\
                 index=False)

#### SOFA window: (6,3)

In [87]:
Q = """
with ids as (select * from train_ids union select * from val_ids)
select e.*, s.t_suspicion, s.t_sofa, s.t_sepsis_min, s.t_sepsis_max
from extracted_data e 
inner join ids on e.subject_id = ids.subject_id
  inner join sepsis_cohort_time_blood_sensitivity_63 s 
  on e.icustay_id = s.icustay_id order by subject_id, charttime"""
df_sepsis = execute_query(Q, con, schema_name)

In [88]:
df_sepsis = df_sepsis.drop_duplicates()

In [89]:
len(df_sepsis[df_sepsis[['icustay_id', 'charttime']].duplicated()])

11

In [90]:
df_sepsis_merge = df_sepsis.merge(df_care, on=['icustay_id', 'charttime'], how='inner')

Next we again remap the ids

In [91]:
print('Merging new icustay_ids')

df_newid2 = pd.merge(df_icustay_ids, df_sepsis_merge, on='icustay_id')

print('Merging new hadm_ids')

df_newid2 = pd.merge(df_hadm_ids, df_newid2, on='hadm_id')

print('Merging new subject_ids')

df_newid2 = pd.merge(df_subject_ids, df_newid2, on='subject_id')

Merging new icustay_ids
Merging new hadm_ids
Merging new subject_ids


In [92]:
df_newid2.drop(['subject_id', 'hadm_id', 'icustay_id'], axis = 1, inplace=True)

df_newid2.rename(columns={"subject_id_random": "subject_id", "hadm_id_random": "hadm_id",\
                          "icustay_id_random":"icustay_id"}, inplace=True)

We move the static columns to the start of the dataframe

In [93]:
new_columns = static_info + (df_newid2.columns.drop(static_info).tolist())

df_newid2 = df_newid2[new_columns]

df_sepsis = df_newid2.sort_values(['subject_id', 'charttime'])

We note that by reducing the SOFA window from (48,24) to (6,3), there is a large reduction of patients identified with sepsis, from around 6000 to around 4000.

In [94]:
len(df_sepsis[~df_sepsis.t_sepsis_min.isnull()].icustay_id.unique())

4134

In [95]:
len(df_sepsis.icustay_id.unique())

13971

Exclude patients based on length of stay and print summary for future exclusions

In [96]:
df_sepsis = df_sepsis[(df_sepsis['outtime'] <= df_sepsis['intime']+pd.Timedelta(days=20)) & \
             (df_sepsis['outtime'] >= df_sepsis['intime']+pd.Timedelta(hours=4))]

In [97]:
len(df_sepsis.icustay_id.unique())

13704

In [98]:
# We want to filter out patients without any vital sign data

vitals = ['heart_rate', 'nbp_sys', 'nbp_dias', 'nbp_mean', 'abp_sys',
       'abp_dias', 'abp_mean', 'temp_celcius', 'o2sat', 'resp_rate']

df_vitals = df_sepsis[['icustay_id']+vitals].groupby(by='icustay_id', as_index=False).max()
df_max = df_vitals.drop(['icustay_id'], axis=1).max(axis=1)
keep_ids = df_vitals[~df_max.isnull()].icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [99]:
len(df_sepsis.icustay_id.unique())

13604

Next we keep only the index hospital admission of each patient

In [100]:
df_info = df_sepsis[['subject_id', 'icustay_id', 'intime']].drop_duplicates()
df_info.sort_values('intime')
df_info2 = df_info.groupby(by='subject_id', as_index=False).first()
keep_ids = df_info2.icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [101]:
sepsis_summary(df_sepsis)

There are 11270 ICU stays after LOS, vital sign and stay exclusions
---------------
 Looking at t_suspicion
We will be keeping 1059 ICU stays and removing 2091

---------------
 Looking at t_sofa
We will be keeping 783 ICU stays and removing 2367

---------------
 Looking at t_sepsis_min
We will be keeping 737 ICU stays and removing 2413

---------------
 Looking at t_sepsis_max
We will be keeping 1105 ICU stays and removing 2045



In [102]:
df_sepsis.to_csv('../../data/raw/blood_only_sensitivity_6_3.csv',\
                 index=False)

## Other sensitivity analysis

We also check sensitivity on blood cultures (whether to use all cultures or not), SOFA score (whether to use absolute value or change in score) and CNS component (whether it adds unnecessary noise). For the other sensitivity analysis, we will use a SOFA window of (24,12) for all of them.

### Sepsis - Other cultures 

Now we extract the sepsis onset time that we get by using a more general list of cultures taken rather just than those relating to blood cultures. We expect that this may increase the sensitivity at a much decreased specificity. This is slightly different to the approach taken in some other papers where all culture types were used.

In [103]:
Q = """
with ids as (select * from train_ids union select * from val_ids)
select e.*, s.t_suspicion, s.t_sofa, s.t_sepsis_min, s.t_sepsis_max
from extracted_data e 
inner join ids on e.subject_id = ids.subject_id
  inner join sepsis_cohort_time_cultures_2412 s 
  on e.icustay_id = s.icustay_id order by subject_id, charttime"""
df_sepsis = execute_query(Q, con, schema_name)

In [104]:
df_sepsis = df_sepsis.drop_duplicates()

In [105]:
len(df_sepsis[df_sepsis[['icustay_id', 'charttime']].duplicated()])

11

In [106]:
df_sepsis_merge = df_sepsis.merge(df_care, on=['icustay_id', 'charttime'], how='inner')

Next we again remap the ids

In [107]:
print('Merging new icustay_ids')

df_newid2 = pd.merge(df_icustay_ids, df_sepsis_merge, on='icustay_id')

print('Merging new hadm_ids')

df_newid2 = pd.merge(df_hadm_ids, df_newid2, on='hadm_id')

print('Merging new subject_ids')

df_newid2 = pd.merge(df_subject_ids, df_newid2, on='subject_id')

Merging new icustay_ids
Merging new hadm_ids
Merging new subject_ids


In [108]:
df_newid2.drop(['subject_id', 'hadm_id', 'icustay_id'], axis = 1, inplace=True)

df_newid2.rename(columns={"subject_id_random": "subject_id", "hadm_id_random": "hadm_id",\
                          "icustay_id_random":"icustay_id"}, inplace=True)

We move the static columns to the start of the dataframe

In [109]:
new_columns = static_info + (df_newid2.columns.drop(static_info).tolist())

df_newid2 = df_newid2[new_columns]

df_sepsis = df_newid2.sort_values(['subject_id', 'charttime'])

We see that with the inclusion of all cultures, a greater number of people have been flagged as having sepsis (compared with the 5961 of the blood cultures)

In [110]:
len(df_sepsis[~df_sepsis.t_sepsis_min.isnull()].icustay_id.unique())

5981

In [111]:
len(df_sepsis.icustay_id.unique())

13971

Exclude patients based on length of stay and print summary for future exclusions

In [112]:
df_sepsis = df_sepsis[(df_sepsis['outtime']<=df_sepsis['intime']+pd.Timedelta(days=20)) & \
             (df_sepsis['outtime']>=df_sepsis['intime']+pd.Timedelta(hours=4))]

In [113]:
len(df_sepsis.icustay_id.unique())

13704

In [114]:
# We want to filter out patients without any vital sign data

vitals = ['heart_rate', 'nbp_sys', 'nbp_dias', 'nbp_mean', 'abp_sys',
       'abp_dias', 'abp_mean', 'temp_celcius', 'o2sat', 'resp_rate']

df_vitals = df_sepsis[['icustay_id']+vitals].groupby(by='icustay_id', as_index=False).max()
df_max = df_vitals.drop(['icustay_id'], axis=1).max(axis=1)
keep_ids = df_vitals[~df_max.isnull()].icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [115]:
len(df_sepsis.icustay_id.unique())

13604

Next we keep only the index hospital admission of each patient

In [116]:
df_info = df_sepsis[['subject_id', 'icustay_id', 'intime']].drop_duplicates()
df_info.sort_values('intime')
df_info2 = df_info.groupby(by='subject_id', as_index=False).first()
keep_ids = df_info2.icustay_id.unique()
df_sepsis = df_sepsis[df_sepsis.icustay_id.isin(keep_ids)]

In [117]:
sepsis_summary(df_sepsis)

There are 11270 ICU stays after LOS, vital sign and stay exclusions
---------------
 Looking at t_suspicion
We will be keeping 1426 ICU stays and removing 3203

---------------
 Looking at t_sofa
We will be keeping 1086 ICU stays and removing 3543

---------------
 Looking at t_sepsis_min
We will be keeping 724 ICU stays and removing 3905

---------------
 Looking at t_sepsis_max
We will be keeping 1788 ICU stays and removing 2841



In [118]:
df_sepsis.to_csv('../../data/raw/other_cultures_sensitivity_24_12.csv',\
                 index=False)

In [119]:
con.close()